In [1]:
import math
import random
import torch as t
import numpy as np
import torch.nn.functional as f
import matplotlib.pyplot as plt
import torch.optim as opt
from torch.autograd import Variable as Var


def valuation(vv_1, vv_2, x):
    v_1 = vv_1 * (x - x ** 2 / 2)
    v_2 = vv_2 * (1 - x)
    return v_1, v_2


def loss_function(value):
    return -value




In [2]:
class Lambda(t.nn.Module):
    def __init__(self, num_input, num_hidden, num_output):

        super(Lambda, self).__init__()
        self.u_off = Var(t.Tensor([1.0]), requires_grad=True)
        self.u_off = t.nn.Parameter(self.u_off).cuda()
        self.u_def = Var(t.Tensor([3.0]), requires_grad=True)
        self.u_def = t.nn.Parameter(self.u_def).cuda()

        self.hidden = t.nn.Linear(num_input, num_hidden)
        self.hidden_plus = t.nn.Linear(num_hidden, num_hidden)
        self.hidden_deep = t.nn.Linear(num_hidden, num_hidden)
        self.output = t.nn.Linear(num_hidden, num_output)

        self.hidden = self.hidden.cuda()
        self.hidden_plus = self.hidden_plus.cuda()
        self.hidden_deep = self.hidden_deep.cuda()
        self.output = self.output.cuda()

    def forward(self, x, type_off, type_def):

        h = t.sigmoid(self.hidden(x))
        h = t.relu_(self.hidden_plus(h))
        h = t.sigmoid(self.hidden_deep(h))
        o = self.output(h)

        val_off, val_def = valuation(type_off, type_def, x)
        val_off *= self.u_off
        val_def *= self.u_def

        val_off += o
        val_def += o

        max_off, max_t_off = val_off.max(0)
        max_def, max_t_def = val_def.max(0)

        val_sum = val_off + val_def
        max_val, t_end = val_sum.max(0)
        end_val_off = val_off[int(t_end)]
        end_val_def = val_def[int(t_end)]

        pay_off = min((max_def - end_val_def) / self.u_off, type_off)
        pay_def = min((max_off - end_val_off) / self.u_def, type_def)
        payment = pay_off + pay_def

        return payment, o, x[int(t_end)]




SyntaxError: invalid syntax (<ipython-input-2-16cdf69726e9>, line 5)

In [ ]:
random.seed(2000)
t.manual_seed(256)
time_frame = t.linspace(0, 1, 1000)
time_frame = t.unsqueeze(time_frame, dim=1)
time_frame = Var(time_frame)
time_frame = time_frame.cuda()
lamb = Lambda(1, 100, 1)

print(time_frame)

In [ ]:
optimizer = opt.RMSprop(lamb.parameters(), lr=0.0001, alpha=0.9)
optimizer = opt.SGD(lamb.parameters(), lr=0.01)
# optimizer = opt.Adam(lamb.parameters(), lr=0.0001, betas=(0.9, 0.99))


batch_size = 64
batch_length = int(math.sqrt(batch_size))
t_off = np.linspace(0, 400, batch_length)
t_def = np.linspace(0, 15, batch_length)
index_train = []
for i in range(batch_length):
    for j in range(batch_length):
        index_train.append([t_off[i], t_def[j]])
        
        

echo = 10000

In [ ]:
print(index_train)

In [ ]:



for i in range(echo):

    # offender_types = []
    # defender_types = []
    loss_sum = 0
    """
    for j in range(batch_size):
        offender_types.append(random.randint(0, 400))
        defender_types.append(random.randint(0, 15))
    """
    for j in range(batch_size):
        p, curve, te = lamb(time_frame, index_train[j][0], index_train[j][1])
        loss_sum += p

    loss = loss_function(loss_sum / batch_size)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 500 == 0:
        # offender_types = sum(offender_types) / batch_size
        # defender_types = sum(defender_types) / batch_size
        loss_sum = loss_sum[0].data / batch_size
        # print(offender_types, defender_types, loss_sum)
        print("%.5f" % loss_sum.item())

        plt.cla()
        plt.axvline(te.cpu().data)
        plt.plot(time_frame.cpu().data.numpy(), curve.cpu().data.numpy())
        plt.text(
            0.5,
            0,
            "Loss=%.4f" % (loss.cpu().data.numpy()),
            fontdict={"size": 20, "color": "red"},
        )
        plt.pause(0.03)